# Machine Learning I, Programming Exercise 6
You will probably find that the tasks in this notebook are considerably more difficult than those in the previous mandatory coding exercises. That is because it will examine new concepts that are only touched upon briefly in the lecture. If you get stuck at some point or find that the task descriptions are unclear, please ask your questions in the "Exercise Questions" channel of the Mattermost and tag it with `@tmichels` so that I will be notified.

If you haven't worked with PyTorch yet, you might want to check out the solution for the previous programming exercise, as it contains a brief introduction to PyTorch and the convolution operation.

## 1. Neural Network Classifiers (OPTIONAL, but it might be helpful to read the exercise text)
In this exercise, we want to implement a simple neural network classifier in PyTorch. In order to visualize similarities and/or differences to the previous learning models (SVM, kNN, etc.), we will apply our classifier to the same toy datasets. However, we will consider the multi-class case with $k \geq 2$ different classes right from the start. More formally, we define a neural network for classification as a function $f: \mathcal{X} \to \mathbb{R}^k$ and we predict a class label $\hat{y} \in \mathcal{Y} = \{1, \ldots, k\}$ to an input $\mathbf{x} \in \mathcal{X}$ in the following way:
\begin{equation*}
    \hat{y} = \operatorname{arg\ max}_{y \in \{1, \ldots, k\}} f(\mathbf{x})_y,
\end{equation*}
that is we consider the class with maximal *score*. 

As you have learned in the lecture already, neural networks usually don't use the hinge loss for training, but rather the log loss which is defined as
\begin{equation*}
    l(f(\mathbf{x}), y) = \log\left(1 + e^{-yf(\mathbf{x})}\right).
\end{equation*}
However, using the log loss still makes sense only in the binary classification setting, since it requires that the output of $f$ is scalar. Luckily, one can extend the binary log loss to the case where the number of classes $k\geq2$:
\begin{equation*}
    \ell(f(\mathbf{x}), y) = -\log(\operatorname{softmax}(f(\mathbf{x}))_y),
\end{equation*}
where
\begin{equation*}
    \operatorname{softmax}(\mathbf{z})_i := \frac{e^{z_i}}{\sum_{j=1}^{k} e^{z_j}}
\end{equation*}
and $y \in \mathcal{Y}$ is the true label for input $\mathbf{x} \in \mathcal{X}$. This loss is usually called *cross entropy* loss and its functionality is implemented in the [`torch.nn.CrossEntropyLoss`](https://pytorch.org/docs/stable/generated/torch.nn.CrossEntropyLoss.html) module.

It is not straightforward to see that $\ell$ is really an extension to the log loss from the lecture, so let's have a closer look at the case where $k=2$. Contrary to the lecture, we assume class labels $y \in \{1, 2\}$ instead of $\tilde{y} \in \{-1, 1\}$. First, we examine the softmax function for $\mathbf{z} = f(\mathbf{x}) \in \mathbb{R}^2$:
\begin{equation*}
    \operatorname{softmax}(\mathbf{z})_1 = \frac{e^{z_1}}{e^{z_1} + e^{z_2}} = \frac{e^{z_1}}{e^{z_1}(1 + e^{z_2-z_1})} = \frac{1}{1 + e^{-(z_1-z_2)}} = \sigma(z_1 - z_2)
\end{equation*}
and by the same calculation
\begin{equation*}
    \operatorname{softmax}(\mathbf{z})_2 = \sigma(z_2 - z_1).
\end{equation*}
Now consider the case that the true label $y=1$. We can directly calculate the cross entropy loss as
\begin{align*}
    \ell(\mathbf{z}, y) &= -\log(\operatorname{softmax}(\mathbf{z})_1)\\
        &= -\log(\sigma(z_1-z_2))\\
        &= -\log\left(\frac{1}{1 + e^{-(z_1-z_2)}}\right)\\
        &= \log\left(1 + e^{-(z_1-z_2)}\right)\\
        &= \log\left(1 + e^{-\tilde{y}(z_1-z_2)}\right)
\end{align*}
for $\tilde{y}=1$. We can also consider the case $y=2$, for which we get by the same calculation that
\begin{equation*}
    \ell(\mathbf{z}, y) = \log\left(1 + e^{-\tilde{y}(z_1-z_2)}\right)
\end{equation*}
for $\tilde{y}=-1$. Hence, we conclude that the cross entropy loss is indeed an extension of the log loss from the lecture. Note that $\mathbf{z} = f(\mathbf{x}) \in \mathbb{R}^k$ is called the *logit* vector in most of the literature.

Well, that should be enough theory for a programming exercise, so let's get started with implementing what we just learned:
1. Implement a fully connected neural network with a specified number of hidden layers of and each with a specified number of neurons. That is, the network class should accept a parameter `hidden_layers`, which specifies both. For example, `hidden_layers=5` should produce a network with one hidden layer that contains 5 neurons. `hidden_layers=[10, 5]` on the other hand would produce two hidden layers, the first of which has 10 neurons and a second one with 5.

   In the PyTorch framework, this is usually achievd by subclassing [`torch.nn.Module`](https://pytorch.org/docs/stable/nn.html#module) and overriding the `forward` method. This method should define how a network input is to be transformed into an output and you can use any of the layers that you have defined in the class' constructor. It should also be possible to pass an activation function to the module which is then used between each fully-connected layer. The output of the final layer should not be passed through an activation function.
   
   **Hint:** You might want to have a look at [`torch.nn.ModuleList`](https://pytorch.org/docs/stable/nn.html#modulelist).


2. PyTorch does not offer any convenience functions for its modules, such as `fit(X, y)` for training on some dataset. Therefore, you'll need to write your own functions for training the model on a given dataset or applying the network to previously unseen data, if you want to use the PyTorch network with the same API as the Scikit-Learn models from earlier exercises. However, unlike with the SVMs that you have implemented from scratch, there is no need to manually compute any gradients, since they can be computed automatically by PyTorch's `autograd` engine which, in fact, implements an extended version of the backpropagation algorithm from the lecture. 

   So, in order to make your PyTorch networks work with the Scikit-Learn API, write a wrapper class that takes an existing PyTorch network and offers the functions `fit(X, y)` for training the network and `predict(X)` for applying the network to previously unseen data. `X` and `y` should be Numpy arrays as usual.
   
   Training should be done via mini-batch stochastic gradient descent, see [`torch.optim.SGD`](https://pytorch.org/docs/stable/optim.html#torch.optim.SGD) for an implementation that you can use. However, instead of sampling a random mini-batch before each update step, it is much more common in neural network training to randomly shuffle the entire dataset before the actual training loop and then process the shuffled data deterministically from start to end. One pass through the entire dataset is called an *epoch* and training is usually not limited by a fixed amount of gradient descent updates but by the number of epochs to perform instead. Note that the step size of the (stochastic) gradient descent algorithm is often called *learning rate* in the context of neural network training.


3. Train your neural network on the toy dataset(s) generated in the code skeleton for a sufficient number of epochs (about 500) and visualize its decision boundary for different activation functions and hidden layer configurations (number of neurons and number of layers).

In [1]:
%matplotlib inline
import numpy as np
import torch
import matplotlib.pyplot as plt
import matplotlib.colors as colors
from sklearn.datasets import make_classification, make_moons
from sklearn.model_selection import train_test_split
from sklearn.base import BaseEstimator, ClassifierMixin


class SimpleNet(torch.nn.Module):
    def __init__(self, activation=None, num_classes=10, in_features=2, hidden_layers=10):
        super(SimpleNet, self).__init__()
        
        # TODO: initialise the network's layers here
        raise NotImplementedError('TODO')

    def forward(self, x):
        # TODO: Apply the layers and activation to the input
        raise NotImplementedError('TODO')

class TorchClassifier(BaseEstimator, ClassifierMixin):
    def __init__(self, torch_model, loss, optimizer_class, device, step_size=0.001, 
                 batch_size=64, epochs=20):
        super(TorchClassifier, self).__init__()
        
        raise NotImplementedError('TODO')
        
    def fit(self, X, y):
        # TODO: Train the network on the given data
        raise NotImplementedError('TODO')


    def predict(self, X):
        # TODO: Predict labels for X using the trained network
        raise NotImplementedError('TODO')
            

# Generate toy data
n = 200
n_classes = 2
X, Y = make_moons(n, noise=0.3, random_state=123456)

#n_classes = 2
#X, Y = make_classification(n, n_features=2, n_redundant=0, n_classes=n_classes,
#                           n_clusters_per_class=2, class_sep=1.2, random_state=12345)
# You can also try a dataset with more than 2 classes
#n_classes = 3
#X, Y = make_classification(n, n_features=2, n_redundant=0, n_classes=n_classes,
#                           n_clusters_per_class=1, class_sep=1.0, random_state=5432)
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.25, random_state=12345)

loss = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD
# Change this to 'cuda' if you can and want to use a GPU for training
# Otherwise, you can completely ignore this parameter
device = torch.device('cpu')  
batch_size=100
epochs = 500
def get_classifier(activation=None, hidden_layers=10, step_size=0.001):
    return TorchClassifier(SimpleNet(activation=activation, hidden_layers=hidden_layers, num_classes=n_classes), 
                           loss, optimizer, device, step_size=step_size, batch_size=batch_size, epochs=epochs)
classifiers = [
    # Add some instances of the neural network here, for example:
    # ('Activation: ReLU, Hidden Neurons: 10, 10',
    #  get_classifier(torch.nn.ReLU(), [10, 10], 0.5)),
]

# Plot decision surface
# First generate grid
res = 200  # Resolution of the grid in cells
x_max, y_max = np.max(X, axis=0)
x_min, y_min = np.min(X, axis=0)
x_min, x_max = x_min - (x_max - x_min) * 0.0625, x_max + (x_max - x_min) * 0.0625
y_min, y_max = y_min - (y_max - y_min) * 0.0625, y_max + (y_max - y_min) * 0.0625
grid_x, grid_y = np.meshgrid(np.linspace(x_min, x_max, res),
                             np.linspace(y_min, y_max, res))
# Get test array from grid
grid_input = np.c_[grid_x.reshape(-1), grid_y.reshape(-1)]

cmap = colors.ListedColormap([[0.0, 0.0, 1.0], [1.0, 0.0, 0.0], [0.0, 1.0, 0.0]])
rows = (len(classifiers)+1)//2
fig, axes = plt.subplots(rows, 2, sharex=True, sharey=True, figsize=(12, 4.5*rows))
for (name, clf), ax in zip(classifiers, axes.ravel()):
    clf.fit(X_train, Y_train)
    score = clf.score(X_test, Y_test)
    train_score = clf.score(X_train, Y_train)
    grid_out = clf.predict(grid_input).reshape(grid_x.shape)

    ax.set_title('%s,\nTrain. Acc.: %.3f, Val. Acc.: %.3f' % (name, train_score, score))
    ax.contourf(grid_x, grid_y, grid_out, alpha=0.5, cmap=plt.cm.brg)
    ax.scatter(X_train[:, 0], X_train[:, 1], c=Y_train, cmap=cmap, edgecolor='k')
plt.tight_layout()
plt.show()
plt.close(fig)

<Figure size 864x0 with 0 Axes>

## 2. LeNet (MANDATORY)
The *LeNet-5* architecture, proposed in 1998 by Yann LeCun et al., was one of the first CNNs and can be seen as the predecessor and source of inspiration of most modern CNN architectures. Most beginner-level tutorials on CNNs include the implementation of a "LeNet-like" network (for example the [official PyTorch tutorials](https://pytorch.org/tutorials/beginner/blitz/neural_networks_tutorial.html)), which differs from the original in quite some ways. In this exercise, however, we would like to re-implement the LeNet-5 architecture as close to the original as it is reasonably possible in a modern framework such as PyTorch. Please read the relevant sections on LeNet-5 from the [paper](http://yann.lecun.com/exdb/publis/pdf/lecun-01a.pdf) by LeCun et al. and implement their architecture with two small changes:
* You don't have to reproduce the complicated connection pattern between feature maps in layer *C3*. But in order to stay faithful to the spirit of original, implement the layer such that half of the filters are connected only to the first three input feature maps and the other half to the last three.
* The output layer (called RBF layer in the paper) with fixed parameters is not too useful if you want to use the network on different datasets. Replace it by a standard fully-connected layer that has as many outputs as there are classes for the specific task.

**Hints:**
* Carefully read the documentation of the [`torch.nn.Conv2d`](https://pytorch.org/docs/stable/generated/torch.nn.Conv2d.html) module to understand what the different parameters of a convolution layer are and what their effect is. You can also check the previous programming exercise for an introduction to convolutions.
* When you put everything together as described in the paper and the two bullet points above, your model should have seven layers and a total of 60550 trainable parameters.

In [2]:
import torch
import torch.nn as nn

class LeNet(torch.nn.Module):
    def __init__(self, activation=None, num_classes=10, in_channels=3):
        super(LeNet, self).__init__()
        self.activation = activation
        
        # TODO: initialise the network's layers here
        pool = torch.nn.AvgPool2d(2, 2)
        self.C1 = torch.nn.Conv2d(in_channels, 6, 5, padding=2)
        self.S2 = torch.nn.Sequential(pool, torch.nn.Conv2d(6, 6, 1, groups=6))
        self.C3 = torch.nn.Conv2d(6, 16, 5, groups=2)
        self.S4 = torch.nn.Sequential(pool, torch.nn.Conv2d(16, 16, 1, groups=16))
        self.C5 = torch.nn.Conv2d(16, 120, 5)
        self.F6 = torch.nn.Linear(120, 84)
        self.out = torch.nn.Linear(84, num_classes) ##ok
        #raise NotImplementedError('TODO')

    def forward(self, x):
        # TODO: Apply the layers and activation to the input
        x = self.activation(self.C1(x))
        x = self.activation(self.S2(x))
        x = self.activation(self.C3(x))
        x = self.activation(self.S4(x))
        x = self.activation(self.C5(x))
        x = self.activation(self.F6(x))
        x = self.out(x)
        return x
        #raise NotImplementedError('TODO')

# TODO: Choose the activation function from the paper
activation = lambda x: 1.7159*torch.tanh(x)
network = LeNet(activation=activation, in_channels=1)
print('Network parameters:', sum(p.numel() for p in network.parameters()))

Network parameters: 60550


## 3. ANN Training (MANDATORY)
In this exercise, we want to use the CNN from part 2 on the so called "[Fashion MNIST](https://github.com/zalandoresearch/fashion-mnist)" dataset. It comes with a training set that contains 60000 gray-scale images of size 28 by 28 pixels and a test set with 10000 images of the same dimensions. Each image belongs to exactly one of the ten classes and each class corresponds to a different piece of clothing (hence the name). Fashion MNIST is intended as a replacement for the original MNIST dataset which contained images of handwritten digits from 0 to 9 with the same dimensionality. As you have probably read in the paper by LeCun et al., the LeNet-5 architecture was originally used for recognizing such handwritten digits.

You don't need to download the dataset manually, since this functionality is already included in the `torchvision` package, please take a look at [`torchvision.datasets.FashionMNIST`](https://pytorch.org/vision/stable/datasets.html#fashion-mnist). However, this class won't load all the images into memory on its own, instead data is only streamed from disk at the moment where you access it. Note that this is actually desired behaviour, since some datasets might not even fit in your main memory, so you would ideally only keep the mini-batch that you are currently processing in memory. This kind of functionailty is implemented in the [`torch.utils.data.DataLoader`](https://pytorch.org/docs/stable/data.html#torch.utils.data.DataLoader) class. It creates an iterator which loops over a shuffled version of the given dataset and returns a `(batch_X, batch_y)` tuple with each iteration. One pass through the iterator corresponds to one epoch of training.

1. Training a neural network can take a considerable amount of time, so it is important to keep track of what is going on during training. For that, write a new training function that trains a neural network using a `DataLoader` as described above. As mentioned in ex. 1, training should be done via mini-batch stochastic gradient descent; see [`torch.optim.SGD`](https://pytorch.org/docs/stable/optim.html#torch.optim.SGD) for an implementation that you can use. However, instead of sampling a random mini-batch before each update step, it is much more common in neural network training to randomly shuffle the entire dataset before the actual training loop and then partition the shuffled data deterministically into mini-batches from start to end. One pass through the entire dataset is called an *epoch* and training is usually not limited by a fixed amount of gradient descent updates but by the number of epochs to perform instead. Note that the step size of the (stochastic) gradient descent algorithm is often called *learning rate* in the context of neural network training. Most of that functionality is already included in the `DataLoader` or `torch.optim.SGD` classes, please consult their documentation for more details.

   Before training, the dataset is usually split into a training and test dataset, or -- like it is the case for FashionMNIST -- the dataset already contains distinct training and test samples. Report the average loss and accuracy (fraction of correctly classified inputs) on the training dataset during each epoch and also compute average loss and accuracy on the test set after each epoch. Your function should return a collection of those values, so that they can be visualized later.

   **Tip:** The `tqdm` package makes it easy to create good-looking progress bars that you can customize with your own texts. To make it work in a Jupyter notebook, you should use the classes from the [`tqdm.notebook`](https://tqdm.github.io/docs/notebook/) module.


2. Train the LeNet network from ex. 2 on the Fashion MNIST dataset for 15 epochs using the function from part 1 with the following parameters:
   * batch size: 64
   * step_size: 0.01
   * 2-norm regularization coefficient (also called "weight decay"): 0.0004 
   * loss function: [`torch.nn.CrossEntropyLoss`](https://pytorch.org/docs/stable/generated/torch.nn.CrossEntropyLoss.html) (see ex. 1 for details)
   
   Your Network should reach at least 86% accuracy on the test set in one of the 15 epochs.
   
   Then use the data returned by the training function to visualize the training procedure. More specifically, create two plots: One should show the average loss on the y-axis and the epoch on the x-axis. Plot the values for training and testing set in the same figure, so that you can easily compare them. You should do the same for the accuracy in each epoch, but in a separate plot.

   **Tip:** Take a look at the [`torch.save`](https://pytorch.org/docs/stable/generated/torch.save.html) function which allows you to save your model after or during training. It might become annoying to re-train the model each time you change a line of code, so you can train it just once and then simply load it from disk afterwards.


3. Given a dataset $D = \{(\mathbf{x}^{(1)}, y^{(1)}), (\mathbf{x}^{(2)}, y^{(2)}), \ldots, (\mathbf{x}^{(n)}, y^{(n)})\} \subseteq \mathcal{X} \times \{1, \ldots, k\}$ and a classifier $f: \mathcal{X} \to \{1, \ldots, k\}$, we define the confusion matrix $C_f$ in the following way:
\begin{equation*}
    C_{ij} := \frac{\lvert \left\{ (\mathbf{x}, y) \in D \mid y=i \land f(\mathbf{x})=j \right\} \rvert}{\lvert \left\{ (\mathbf{x}, y) \in D \mid y=i \right\} \rvert},
\end{equation*}
i.e., $C_{ij}$ is defined as the fraction of data points from $D$ that belong to class $i$ and are classified as $j$ by $f$. Such a matrix offers more information than a simple scalar like accuracy or loss, since it becomes obvious where exactly the classifier makes its mistakes.
  
   After training the LeNet on the Fashion MNIST training dataset, compute, show and interpret the confusion matrix of that classifier on the Fashion MNIST test dataset. You can also compute the confusion matrix after each epoch of training to see how it develops. A mapping from numeric class labels to semantically meaningful class names is available on the [Fashion MNIST website](https://github.com/zalandoresearch/fashion-mnist#labels).
   

4. **(OPTIONAL)** Start experimenting with hyperparameters such as learning rate, batch size, and/or optimization algorithm. Can you improve the test set accuracy compared to the original set of parameters?

In [4]:
%matplotlib inline
import os
import torch
import torchvision
import matplotlib.pyplot as plt

def train_model(network, loss, optimizer, train_iter, val_iter, num_epochs, device='cpu', start_epoch=0,
                checkpoints=False, out_dir=None):
    """
    Train a PyTorch neural network with the specified parameters
    
    :param network: The neural network to train. Should be a subclass of :class:`torch.nn.Module`
    :param loss: The loss function to use. Should take arguments of the form (prediction, true_value)
    :param optimizer: The optimization algorithm that is used for training. Should be a subclass of :class:`torch.optim.Optimizer`
    :param train_iter: A :class:`torch.utils.data.Dataloader` instance that yields mini-batches for training
    :param val_iter: A :class:`torch.utils.data.Dataloader` instance that yields mini-batches for testing
    :param num_epochs: The number of epochs to train the network for
    :param device: The device to use for training. Usually 'cpu' or 'cuda'
    :param start_epoch: The first epoch to start with. This is for the case of resuming training from a checkpoint
    :param checkpoints: Whether to save a checkpoint of the model after each training epoch
    :param out_dir: Directory where the checkpoints will be saved
    
    :returns: The avg. training and test loss and accuracy for each epoch
    """
    for i in range(num_epochs):
        current_batch = next(iter(train_iter))
        X, Y = current_batch
        predictions = network(X)
        print(predictions)
    
    # TODO: Train the model and return the training history
    raise NotImplementedError('TODO')

def plot_history(history):
    # TODO: Plot training and validation curves for avg. loss and accuracy
    raise NotImplementedError('TODO')
    
def confusion_matrix(network, val_iter, num_classes=10, device='cpu'):
    # TODO: Compute the confusion matrix for classifier 'network' and
    # dataset 'val_iter' (given as a DataLoader)
    raise NotImplementedError('TODO')


# Change this to 'cuda' if you can and want to use a GPU for training
# Otherwise, you can completely ignore this parameter
device = torch.device('cpu')
dataset = torchvision.datasets.FashionMNIST
batch_size = 64
num_epochs = 15
num_workers = 0  # You can use multiple processes to load the data. 0 means that everything is done in the same process
step_size = 1e-2
regularization=0.0004  # Contols how much the Frobenius norm of the parameters contributes to the final loss
checkpoints = True  # Save the model after each epoch of training
out_dir = 'models'  # Folder, where your model checkpoints will be saved
load_path = None  # Don't train the model, but load it from the specified file instead
data_dir = 'data'  # Folder that contains the dataset. If it is not present, 
                   # the dataset will be downloaded into that folder

#Load the training and the test dataset and wrap them in a DataLoader
train_dataset = dataset(root=data_dir, train=True, download=True, transform = torchvision.transforms.ToTensor())
train_set =  torch.utils.data.DataLoader(torch.utils.data.BufferedShuffleDataset(train_dataset, train_dataset.__len__()), batch_size)

test_dataset = dataset(root=data_dir, train = False, transform = torchvision.transforms.ToTensor())
test_set =torch.utils.data.DataLoader(torch.utils.data.BufferedShuffleDataset(test_dataset, test_dataset.__len__()), batch_size)
#Instantiate the network, optimizer, etc.
activation = lambda x: 1.7159*torch.tanh(x)
network = LeNet(activation=activation, in_channels=1) #in_channels=1 (as we deal with grayscale images)

optimizer = torch.optim.SGD(network.parameters(), step_size)
loss_function = torch.nn.CrossEntropyLoss()

#### TODO -> Move train function in else (see TODO)

train_model(network, loss_function, optimizer, train_set, test_set, num_epochs, device='cpu', start_epoch=0, checkpoints=False, out_dir=None)

if load_path is not None:
    # TODO: Load model, optimizer state and history 
    pass
else:
    # TODO: Otherwise, we train the model for the specified number of epochs
    # using the train function
    pass

# TODO: Plot the training history
# plot_history(history)

# TODO: And finally compute the confusion matrix
# mat = confusion_matrix(network, val_iter, device=device)
# print('Confusion matrix:', mat)

RuntimeError: mat1 and mat2 shapes cannot be multiplied (7680x1 and 120x84)